In [17]:
#Imports
import os, sys
path_to_package = os.path.abspath(os.path.join('../..'))
if path_to_package not in sys.path:
    sys.path.append(path_to_package)


import scipy.stats as stats
import pandas as pd
import plotly.express as px
import plotly.io as pio
pio.renderers.default = "notebook_connected"

from src.io import *
from src.anova import *
from src.utils import *
from src.visualization import *

In [97]:
#ANOVA_DATASET_PATH = "C:\\Users\\Hector\\Desktop\\code\\sc_regmod\\dataset\\anova"
#ANOVA_CSV_DATASET_PATH = "C:\\Users\\Hector\\Desktop\\code\\sc_regmod\\dataset\\anova_res"
#INDEXES_DATASET_PATH = "C:\\Users\\Hector\\Desktop\\code\\sc_regmod\\dataset\\indexes"

ANOVA_DATASET_PATH = "/home/khaldrem/code/sc_regmod/dataset/anova"
ANOVA_CSV_DATASET_PATH = "/home/khaldrem/code/sc_regmod/dataset/anova_res"
INDEXES_DATASET_PATH = "/home/khaldrem/code/sc_regmod/dataset/indexes"

In [3]:
filepaths = get_filepaths(ANOVA_DATASET_PATH)
dataset = []
for filepath in filepaths:
    filename = get_filename(filepath=filepath)
    data = {}
    data["filepath"] = filepath
    data["filename"] = filename
    data["data"] = read_phylip_file(filepath)
    
    dataset.append(data)

In [4]:
# Get columns size by file
def get_columns_size_by_file(dataset):
    res = []
    for data in dataset:
        d = {}
        d["filename"] = data["filename"]
        d["alignment_length"] = data["data"].get_alignment_length()

        res.append(d)
    
    df = pd.DataFrame(res)

    return df

res_1 = get_columns_size_by_file(dataset)
res_1 = res_1.sort_values(by="alignment_length", ascending=False)



In [5]:
# Figura Largo de cadena vs nombre archivo TODOS
fig = px.bar(res_1, x="filename", y="alignment_length")
fig.update_traces(marker_color='red')
fig.show()

In [6]:
# Figura Largo de cadena vs nombre archivo TOP 50
fig = px.bar(res_1.head(50), x="filename", y="alignment_length")
fig.show()

In [82]:
# SHOW BASE OF INTEREST

def get_bases_data_in_file(filename, dataset):
    bases = {}
    for data in dataset:
        if data["filename"].split("/")[-1] == filename:

            #Open index json file
            f = open(f"{INDEXES_DATASET_PATH}/{filename}.json", "r")
            json_object = json.load(f)
            f.close()

            bases = json_object["anova_data"]["bases"]
    
    res = {
        "key": [],
        "val": []
    }
    for key in bases:
        res["key"].append(key)
        res["val"].append(bases[key])

    return pd.DataFrame.from_dict(res)


def get_percentage_bases(data):
    total = data["val"].sum()
    data["percentage"] = data["val"]/data["val"].sum()



def get_base_info_per_file(filename, dataset):
    res = get_bases_data_in_file(filename, dataset)
    get_percentage_bases(res)

    print(res)

    fig = px.bar(res, x="key", y="val")
    fig.show()

    
#Por todos los archivos mostrar la cantidad de 1 sola base que poseen
#Pq: buscar los archivos que poseen mas cantidad de "-"

def get_info_per_base_all_files(base_interest, dataset):
    res_dict = {"filename": [], "base_values": []}
    res = pd.DataFrame.from_dict(res_dict)

    for data in dataset:
        filename = data['filename'].split('/')[-1]

        #per file open index
        f = open(f"{INDEXES_DATASET_PATH}/{filename}.json", "r")
        json_object = json.load(f)
        f.close()

        #check if that base is in the file
        if base_interest in json_object["anova_data"]["bases"]:
            #Agregar datos al dataframe
            res.loc[len(res)] = [filename, int(json_object["anova_data"]["bases"][base_interest])]


    return res




In [83]:
#Files of interest
# YPL283C YGR296W

get_base_info_per_file("YPL283C", dataset)


  key      val  percentage
0   a  1584331    0.292074
1   t  1411977    0.260300
2   g  1329588    0.245112
3   c  1098520    0.202514


In [89]:
base_interest_df = get_info_per_base_all_files("-", dataset)

In [98]:
base_interest_df = base_interest_df.sort_values(by="base_values", ascending=False)
base_interest_df.head(10)


,filename,base_values
2558,YMR173W,1576912.0
2560,YOL155C,1522949.0
2990,YIR019C,1392686.0
3250,YAR050W,1141329.0
1949,YJL020C,1035410.0
568,YCR089W,727073.0
814,YHR211W,674465.0
175,YDL039C,672043.0
2562,YOL051W,510093.0
1964,YIL169C,504486.0


In [93]:
fig = px.bar(base_interest_df, x="filename", y="base_values")
fig.show()

In [95]:
#Top 50
fig = px.bar(base_interest_df.head(50), x="filename", y="base_values")
fig.show()

In [118]:
# Procesamos los csv con los datos obtenidos del anova
# Primero no diferenciamos por ID
# Por cada archivo, obtenemos por cada columna la suma total 
# Guardamos cada resultado de archivo en un dataframe
# filename, SM300-Efficiency,SM300-Rate,SM300-Lag,SM300-AUC,SM60-Efficiency,SM60-Rate,SM60-Lag,SM60-AUC,Ratio-Efficiency,Ratio-Rate,Ratio-Lag,Ratio-AUC 

def get_anova_results_info_all_files(dataset):
    data_rows = {}

    i = 0
    for data in dataset:
        filename = data["filename"].split("/")[-1]
        #print(filename)
        
        res = pd.read_csv(f"{ANOVA_CSV_DATASET_PATH}/{filename}.csv")
        data_rows[str(i)] = [
            filename,
            res["SM300-Efficiency"].sum(),
            res["SM300-Rate"].sum(),
            res["SM300-Lag"].sum(),
            res["SM300-AUC"].sum(),
            res["SM60-Efficiency"].sum(),
            res["SM60-Rate"].sum(),
            res["SM60-Lag"].sum(),
            res["SM60-AUC"].sum(),
            res["Ratio-Efficiency"].sum(),
            res["Ratio-Rate"].sum(),
            res["Ratio-Lag"].sum(),
            res["Ratio-AUC"].sum()
        ]

        i = i + 1

    res = pd.DataFrame.from_dict(data_rows, orient="index", 
                                columns=["filename", 
                                         "SM300-Efficiency", "SM300-Rate", "SM300-Lag", "SM300-AUC",
                                         "SM60-Efficiency", "SM60-Rate", "SM60-Lag", "SM60-AUC",
                                         "Ratio-Efficiency", "Ratio-Rate", "Ratio-Lag", "Ratio-AUC"])

    

    return res

In [119]:
# ETA 18 seg
anova_res_data = get_anova_results_info_all_files(dataset)

In [117]:
anova_res_data.head()

,filename,SM300-Efficiency,SM300-Rate,SM300-Lag,SM300-AUC,SM60-Efficiency,SM60-Rate,SM60-Lag,SM60-AUC,Ratio-Efficiency,Ratio-Rate,Ratio-Lag,Ratio-AUC
0,YGL125W,34.0,29.0,27.0,36.0,53.0,32.0,27.0,42.0,40.0,36.0,15.0,26.0
